# 청와대 청원 분류 경진대회

## 1. 데이터 살펴보기

### 1) 라이브러리 불러오기

In [16]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import urllib.request
import re
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import os
from tensorflow.keras.datasets import reuters
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
from tqdm import tqdm

---

### 2) 데이터 불러오기

In [17]:
train = pd.read_csv('C:/python/train3.csv', encoding = 'utf-8', index_col='index')
test = pd.read_csv('C:/python/test3.csv', encoding = 'utf-8',index_col='index')
sample_submission = pd.read_csv('C:/python/sub3.csv', encoding = 'utf-8')

In [18]:
train.head()

,category,data
index,,
0,2,신혼부부위한 주택정책 보다 보육시설 늘려주세요.. 국민세금으로 일부를 위한 정책펴지...
1,0,학교이름에 '남자'도 붙여주세요. 울산여자중학교에 재학중인 학생입니다 최근 양성평등...
2,1,"빙상연맹, 대한축구협회등 각종 체육협회의 비리를 철저하게 밝혀주세요.. 최근 동계올..."
3,1,"티비 12세,15세 관람가도 연령확인 의무화 하자.. 제기 에전에 티비를 보다가 잠..."
4,1,무더운 여름철엔 남성들도 시원한 자율복장을 해야. 무더운 여름철에는 남성들도 노넥타...


In [19]:
test.head()

,data
index,
0,소년법 폐지해주세요. 법 아래에서 보호받아야 할 아이들이\n법으로 인해 보호받지 못...
1,국공립 유치원 증설에 관하여. 국공립 유치원 부지 학보와건립및 증설에\n*지역 어린...
2,나경원파면. 나경원의원의 동계올림픽 위원을 파면해 주세요
3,국민위원에가 삼성편만들어요. 삼성에서 11년간 일하고 혈암과 백혈병 진단을 받은 ...
4,"방과후,유치원,어린이집 영어교육을 유지시켜주세요. 저는 아이 셋 키우는 평범한 주부..."


### Category
0 : 인권/성평등  
1 : 문화/예술/체육/언론  
2: 육아/교육

---

## 2. 데이터 전처리

In [20]:
# 불용어 처리
train = train.dropna(how = 'any') # row 또는 col에 any가 1개만 있어도 drop
train['data'] = train['data'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
test['data'] = test['data'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다','을']

C:\Users\시형\AppData\Local\Temp\ipykernel_8924\3688304263.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  train['data'] = train['data'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
C:\Users\시형\AppData\Local\Temp\ipykernel_8924\3688304263.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  test['data'] = test['data'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")


In [21]:
# 한글 형태소 분석를 위한 인스턴스(클래스를 사용하기 위한 메모리 구조) 생성
okt = Okt()

In [22]:
# train 데이터
X_train = []
for sentence,i in zip(train['data'],tqdm(range(len(train['data'])))) :
    temp_X = []
    temp_X = okt.morphs(sentence, stem=True)
    temp_X = [word for word in temp_X if not word in stopwords]
    X_train.append(temp_X)

100%|███████████████████████████████████████████████████████████████████████████▉| 39991/39992 [21:45<00:00, 30.63it/s]


In [23]:
# test 데이터
X_test = []
for sentence in test['data']:
    temp_X = []
    temp_X = okt.morphs(sentence, stem=True) # stem=True : 원형 글자 바꾸기
    temp_X = [word for word in temp_X if not word in stopwords]
    X_test.append(temp_X)

#### okt.morphs
텍스트를 형태소 단위로 나눔
#### okt.morphs
텍스트에서 명사만 추출
#### okt.phrases
텍스트의 어절을 추출

---

### 패딩(Padding)

병렬 연산을 위해 여러 문장의 길이를 임의로 동일하게 맞춰주는 작업

#### 1) 단어 집합 생성 및 정수 인코딩 수행

In [24]:
# text를 정수로 변환해주는 Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

In [25]:
vocab_size = 30000
tokenizer = Tokenizer(vocab_size) 
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

#### 2) keras 패딩 수행

In [26]:
max_len = 500
X_train = pad_sequences(X_train, maxlen = max_len)
X_test = pad_sequences(X_test, maxlen = max_len)

In [27]:
y_train = to_categorical(train['category'])

---

## 3. 모델 생성 및 훈련

In [29]:
model = Sequential()
model.add(Embedding(vocab_size, 120))
model.add(LSTM(120))
model.add(Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

history = model.fit(X_train, y_train, batch_size=128, epochs=5)

Epoch 1/5
313/313 [==============================] - 1062s 3s/step - loss: 0.5641 - acc: 0.7813
Epoch 2/5
313/313 [==============================] - 1018s 3s/step - loss: 0.3045 - acc: 0.8940
Epoch 3/5
313/313 [==============================] - 1026s 3s/step - loss: 0.2403 - acc: 0.9146
Epoch 4/5
313/313 [==============================] - 1035s 3s/step - loss: 0.2006 - acc: 0.9293
Epoch 5/5
313/313 [==============================] - 1031s 3s/step - loss: 0.1776 - acc: 0.9367


In [35]:
y_pred = model.predict(X_test)
sample_submission['category'] = y_pred
sample_submission.to_csv('C:/python/sub3.csv', encoding='utf-8')

157/157 [==============================] - 13s 85ms/step


ValueError: Wrong number of items passed 3, placement implies 1